## Dashboard Interactivo del Modelo Mundell-Fleming

Este notebook contiene el código para generar un dashboard interactivo del modelo Mundell-Fleming. Permite visualizar los efectos de políticas fiscales, monetarias y shocks cambiarios en una economía abierta con tipo de cambio flexible.

In [ ]:
# 1. Importación de Librerías
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

In [ ]:
# 2. Configuración del Modelo y Estilo

# Parámetros económicos base del modelo
MODELO_PARAMS = {
    # Curva IS (Mercado de Bienes) -> Y = C(Y-T) + I(i) + G + XN(e)
    # Simplificado a: Y = k * (A_0 - d*i + g*e)
    # Curva DD (Relación Y-e en equilibrio IS-LM): e = DD_I + DD_P * Y
    "DD_I": 5.0,      # Intercepto de la curva DD
    "DD_P": 0.8,      # Pendiente de la curva DD

    # Curva LM (Mercado de Dinero) -> M/P = L(i, Y) = k*Y - h*i
    # Curva AA (Relación Y-e en equilibrio UIP y LM): e = AA_I - AA_P * Y
    "AA_I": 15.0,     # Intercepto de la curva AA
    "AA_P": 1.0,      # Pendiente de la curva AA

    # Curva UIP (Paridad de Tasas de Interés): i = i* + (E^e - e)/e
    # Simplificado a: e = IP_I - IP_P * i
    "IP_I": 20.0,     # Intercepto (relacionado con i* y E^e)
    "IP_P": 1.5,      # Pendiente (sensibilidad a la tasa de interés)

    # Mercado de Dinero: M/P = k*Y - h*i
    "MM_K": 0.8,      # Sensibilidad de la demanda de dinero al ingreso (Y)
    "MM_H": 4.0,      # Sensibilidad de la demanda de dinero a la tasa de interés (i)

    # Mercado de Bienes (Cruz Keynesiana): DA = C + I + G + XN
    # Simplificado a: DA = A_0 + mpc*Y
    "GM_SLOPE": 0.6   # Propensión marginal a gastar (similar a mpc)
}

# Paleta de colores para los gráficos
COLORES = {
    'DD': 'darkgreen',
    'AA': 'crimson',
    'IP': 'darkblue',
    'MD': 'orange',      # Demanda de Dinero (L)
    'MS': 'slateblue',   # Oferta de Dinero (M/P)
    'DA': 'teal',        # Demanda Agregada
    'EQ1': 'black',      # Punto de equilibrio inicial
    'EQ2': 'red',        # Punto de equilibrio final
    'AJUSTE': 'lightcoral',
    'FLECHA': 'darkmagenta'
}

In [ ]:
# 3. Función de Cálculo del Modelo

def calcular_equilibrio(params, shock_fiscal=0, shock_monetario=0, shock_cambiario=0):
    """Calcula los valores de equilibrio del modelo Mundell-Fleming."""
    # Parámetros iniciales
    dd_i_base, dd_p = params['DD_I'], params['DD_P']
    aa_i_base, aa_p = params['AA_I'], params['AA_P']
    ip_i_base, ip_p = params['IP_I'], params['IP_P']
    mm_k, mm_h = params['MM_K'], params['MM_H']
    gm_slope = params['GM_SLOPE']

    # --- Equilibrio Inicial (EQ1) ---
    y_eq1 = (aa_i_base - dd_i_base) / (dd_p + aa_p)
    e_eq1 = dd_i_base + dd_p * y_eq1
    i_eq1 = (ip_i_base - e_eq1) / ip_p
    ms_eq1 = mm_k * y_eq1 - mm_h * i_eq1 # M/P de equilibrio
    da_int_1 = y_eq1 * (1 - gm_slope) # Intercepto de la Demanda Agregada

    # --- Aplicar Shocks y Calcular Equilibrio Final (EQ2) ---
    dd_i_final = dd_i_base + shock_fiscal
    aa_i_final = aa_i_base + shock_monetario + shock_cambiario
    ip_i_final = ip_i_base + shock_cambiario

    y_eq2 = (aa_i_final - dd_i_final) / (dd_p + aa_p)
    e_eq2 = dd_i_final + dd_p * y_eq2
    i_eq2 = (ip_i_final - e_eq2) / ip_p
    ms_eq2 = ms_eq1 + shock_monetario * mm_h # El shock en AA se traduce a un cambio en M/P
    da_int_2 = y_eq2 * (1 - gm_slope)

    return {
        'eq1': {'y': y_eq1, 'e': e_eq1, 'i': i_eq1, 'ms': ms_eq1, 'da_int': da_int_1},
        'eq2': {'y': y_eq2, 'e': e_eq2, 'i': i_eq2, 'ms': ms_eq2, 'da_int': da_int_2},
        'params_final': {'dd_i': dd_i_final, 'aa_i': aa_i_final, 'ip_i': ip_i_final}
    }

In [ ]:
# 4. Función Principal del Dashboard

def dashboard_mundell_fleming(shock_fiscal, shock_monetario, shock_cambiario):
    """Crea y actualiza el dashboard de 4 cuadrantes."""
    
    # Calcula los equilibrios
    data = calcular_equilibrio(MODELO_PARAMS, shock_fiscal, shock_monetario, shock_cambiario)
    eq1, eq2 = data['eq1'], data['eq2']
    params_final = data['params_final']
    p = MODELO_PARAMS

    # Crear figura y ejes para los 4 cuadrantes
    fig, axs = plt.subplots(2, 2, figsize=(14, 12))
    fig.suptitle('Dashboard del Modelo Mundell-Fleming', fontsize=20, y=0.95)
    ((ax1, ax2), (ax3, ax4)) = axs

    # Rangos para los ejes
    Y_range = np.linspace(0, 25, 100)
    I_range = np.linspace(0, 15, 100)
    M_range = np.linspace(-40, 40, 100)

    # --- Cuadrante 1: Mercado Cambiario (UIP) ---
    ax1.set_title('1. Mercado Cambiario (UIP)', fontsize=14)
    ax1.plot(I_range, p['IP_I'] - p['IP_P'] * I_range, color=COLORES['IP'], label='IP (i*, Eᵉ)')
    ax1.plot(I_range, params_final['ip_i'] - p['IP_P'] * I_range, color=COLORES['IP'], linestyle='--', label='IP\'')
    ax1.plot(eq1['i'], eq1['e'], 'o', color=COLORES['EQ1'], markersize=8, label='Eq-1')
    ax1.plot(eq2['i'], eq2['e'], 'o', color=COLORES['EQ2'], markersize=8, label='Eq-2')
    ax1.vlines(eq2['i'], 0, eq2['e'], color=COLORES['AJUSTE'], linestyle=':')
    ax1.hlines(eq2['e'], 0, eq2['i'], color=COLORES['AJUSTE'], linestyle=':')
    ax1.set(xlabel='Tasa de Interés (i)', ylabel='Tipo de Cambio (e)', xlim=(0, 15), ylim=(-5, 30))
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # --- Cuadrante 2: Equilibrio DD-AA ---
    ax2.set_title('2. Equilibrio DD-AA', fontsize=14)
    ax2.plot(Y_range, p['DD_I'] + p['DD_P'] * Y_range, color=COLORES['DD'], label='DD')
    ax2.plot(Y_range, params_final['dd_i'] + p['DD_P'] * Y_range, color=COLORES['DD'], linestyle='--', label='DD\'')
    ax2.plot(Y_range, p['AA_I'] - p['AA_P'] * Y_range, color=COLORES['AA'], label='AA')
    ax2.plot(Y_range, params_final['aa_i'] - p['AA_P'] * Y_range, color=COLORES['AA'], linestyle='--', label='AA\'')
    ax2.plot(eq1['y'], eq1['e'], 'o', color=COLORES['EQ1'], markersize=8, label='Eq-1')
    ax2.plot(eq2['y'], eq2['e'], 'o', color=COLORES['EQ2'], markersize=8, label='Eq-2')
    ax2.vlines(eq2['y'], 0, eq2['e'], color=COLORES['AJUSTE'], linestyle=':')
    ax2.hlines(eq2['e'], 0, eq2['y'], color=COLORES['AJUSTE'], linestyle=':')
    ax2.set(xlabel='Producción (Y)', ylabel='Tipo de Cambio (e)', xlim=(0, 25), ylim=(-5, 30))
    ax2.legend()
    ax2.grid(True, alpha=0.3)

    # --- Cuadrante 3: Mercado de Dinero ---
    ax3.set_title('3. Mercado de Dinero', fontsize=14)
    ax3.plot(M_range, (p['MM_K']/p['MM_H'])*eq1['y'] - (1/p['MM_H'])*M_range, color=COLORES['MD'], label=f'L(Y₁)')
    ax3.plot(M_range, (p['MM_K']/p['MM_H'])*eq2['y'] - (1/p['MM_H'])*M_range, color=COLORES['MD'], linestyle='--', label=f'L(Y₂)')
    ax3.vlines(eq1['ms'], 0, 15, color=COLORES['MS'], label='Mˢ/P')
    ax3.vlines(eq2['ms'], 0, 15, color=COLORES['MS'], linestyle='--', label='Mˢ/P\'')
    ax3.plot(eq1['ms'], eq1['i'], 'o', color=COLORES['EQ1'], markersize=8, label='Eq-1')
    ax3.plot(eq2['ms'], eq2['i'], 'o', color=COLORES['EQ2'], markersize=8, label='Eq-2')
    ax3.vlines(eq2['ms'], 0, eq2['i'], color=COLORES['AJUSTE'], linestyle=':')
    ax3.hlines(eq2['i'], -40, eq2['ms'], color=COLORES['AJUSTE'], linestyle=':')
    ax3.set(xlabel='Saldos Reales (M/P)', ylabel='Tasa de Interés (i)', xlim=(-40, 40), ylim=(0, 15))
    ax3.legend()
    ax3.grid(True, alpha=0.3)

    # --- Cuadrante 4: Mercado de Bienes (Cruz Keynesiana) ---
    ax4.set_title('4. Mercado de Bienes', fontsize=14)
    ax4.plot(Y_range, Y_range, color='gray', label='Y=DA') # Línea de 45 grados
    ax4.plot(Y_range, eq1['da_int'] + p['GM_SLOPE'] * Y_range, color=COLORES['DA'], label='DA₁')
    ax4.plot(Y_range, eq2['da_int'] + p['GM_SLOPE'] * Y_range, color=COLORES['DA'], linestyle='--', label='DA₂')
    ax4.plot(eq1['y'], eq1['y'], 'o', color=COLORES['EQ1'], markersize=8, label='Eq-1')
    ax4.plot(eq2['y'], eq2['y'], 'o', color=COLORES['EQ2'], markersize=8, label='Eq-2')
    ax4.vlines(eq2['y'], 0, eq2['y'], color=COLORES['AJUSTE'], linestyle=':')
    ax4.hlines(eq2['y'], 0, eq2['y'], color=COLORES['AJUSTE'], linestyle=':')
    ax4.set(xlabel='Producción (Y)', ylabel='Demanda Agregada (DA)', xlim=(0, 25), ylim=(0, 25))
    ax4.legend()
    ax4.grid(True, alpha=0.3)

    # Ajustar espaciado y mostrar
    plt.tight_layout(rect=[0, 0, 1, 0.93])
    plt.show()


In [ ]:
# 5. Creación de la Interfaz Interactiva

style = {'description_width': 'initial'}
interact(
    dashboard_mundell_fleming,
    shock_fiscal=widgets.FloatSlider(min=-5, max=5, step=0.5, value=0, description='Shock Fiscal (ΔG)', style=style),
    shock_monetario=widgets.FloatSlider(min=-5, max=5, step=0.5, value=0, description='Shock Monetario (ΔM)', style=style),
    shock_cambiario=widgets.FloatSlider(min=-5, max=5, step=0.5, value=0, description='Shock Cambiario (ΔEᵉ)', style=style)
);